## 0. Torch Distributions
https://pytorch.org/docs/stable/distributions.html

## 1. IMU Noise Model

In [ ]:
import torch
import torch.distributions as D
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# imu_parameters = {
#     'gyroscope_bias_correlation_time': 1000.0,    # tau_g
#     'gyroscope_noise_density': 0.00018665,        # sigma_g
#     'gyroscope_random_walk': 0.000038785,         # sigma_b_g
#     'gyroscope_turn_on_bias_sigma': 0.0087,       # sigma_bon_g
#     'accelerometer_bias_correlation_time': 300.0, # tau_a
#     'accelerometer_noise_density': 0.00186,       # sigma_a
#     'accelerometer_random_walk': 0.006,           # sigma_b_a
#     'accelerometer_turn_on_bias_sigma': 0.196     # sigma_bon_a
# }

# low noise
imu_parameters = {
    'gyroscope_bias_correlation_time': 1000.0,  # tau_g
    'gyroscope_noise_density': 0.00001,           # sigma_g
    'gyroscope_random_walk': 0.000001,           # sigma_b_g
    'gyroscope_turn_on_bias_sigma': 0.001,     # sigma_bon_g
    'accelerometer_bias_correlation_time': 300.0, # tau_a
    'accelerometer_noise_density': 0.0001,        # sigma_a
    'accelerometer_random_walk': 0.001,       # sigma_b_a
    'accelerometer_turn_on_bias_sigma': 0.1   # sigma_bon_a
}

# Initialize biases with turn-on bias using torch distributions
normal_dist = D.Normal(torch.tensor(0.0, device=device), torch.tensor(1.0, device=device))
gyroscope_bias = imu_parameters['gyroscope_turn_on_bias_sigma'] * normal_dist.sample((3,)).to(device)
accelerometer_bias = imu_parameters['accelerometer_turn_on_bias_sigma'] * normal_dist.sample((3,)).to(device)
print(f" init gyro bias: {gyroscope_bias}")
print(f" init accel bias: {accelerometer_bias}")

# Time parameters
dt = torch.tensor(0.005, device=device)  # time step
t_end = 10  # total simulation time
time_steps = torch.arange(0, t_end, dt, device=device)

# Storage for results
gyro_noise = torch.zeros((len(time_steps), 3), device=device)
accel_noise = torch.zeros((len(time_steps), 3), device=device)
gyro_noise[0] = gyroscope_bias
accel_noise[0] = accelerometer_bias

# Precompute constants (tau and sigma values)
tau_g = torch.tensor(imu_parameters['gyroscope_bias_correlation_time'], device=device)
tau_a = torch.tensor(imu_parameters['accelerometer_bias_correlation_time'], device=device)

sigma_g_d = 1 / torch.sqrt(dt) * imu_parameters['gyroscope_noise_density']
sigma_b_g = imu_parameters['gyroscope_random_walk']
sigma_b_g_d = torch.sqrt(-sigma_b_g**2 * tau_g / 2.0 * (torch.exp(-2.0 * dt / tau_g) - 1.0))
phi_g_d = torch.exp(-1.0 / tau_g * dt)

sigma_a_d = 1 / torch.sqrt(dt) * imu_parameters['accelerometer_noise_density']
sigma_b_a = imu_parameters['accelerometer_random_walk']
sigma_b_a_d = torch.sqrt(-sigma_b_a**2 * tau_a / 2.0 * (torch.exp(-2.0 * dt / tau_a) - 1.0))
phi_a_d = torch.exp(-1.0 / tau_a * dt)

for idx in range(1, len(time_steps)):
    # Update gyroscope bias and noise (vectorized for all 3 axes)
    gyroscope_bias = phi_g_d * gyroscope_bias + sigma_b_g_d * normal_dist.sample((3,))#.to(device)
    gyro_noise[idx] = gyroscope_bias + sigma_g_d * normal_dist.sample((3,))#.to(device)

    # Update accelerometer bias and noise (vectorized for all 3 axes)
    accelerometer_bias = phi_a_d * accelerometer_bias + sigma_b_a_d * normal_dist.sample((3,))#.to(device)
    accel_noise[idx] = accelerometer_bias + sigma_a_d * normal_dist.sample((3,))#.to(device)

# Convert back to CPU for plotting
gyro_noise = gyro_noise.cpu().numpy()
accel_noise = accel_noise.cpu().numpy()
time_steps = time_steps.cpu().numpy()

# Plotting the noise for gyroscope and accelerometer
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Gyroscope noise
axs[0].plot(time_steps, gyro_noise[:, 0], label='Gyro X')
axs[0].plot(time_steps, gyro_noise[:, 1], label='Gyro Y')
axs[0].plot(time_steps, gyro_noise[:, 2], label='Gyro Z')
axs[0].set_title('Gyroscope Noise Over Time')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Noise')
axs[0].legend()

# Accelerometer noise
axs[1].plot(time_steps, accel_noise[:, 0], label='Accel X')
axs[1].plot(time_steps, accel_noise[:, 1], label='Accel Y')
axs[1].plot(time_steps, accel_noise[:, 2], label='Accel Z')
axs[1].set_title('Accelerometer Noise Over Time')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Noise')
axs[1].legend()

plt.tight_layout()
plt.show()

## 2. Mag Noise Model

In [ ]:
import torch
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# noisy
mag_parameters = {
   'mag_bias_correlation_time': 600.0,    # tau_m
   'mag_noise_density': 0.0004,           # sigma_m
   'mag_random_walk': 0.0000064,            # sigma_b_m
}

# low noise
# mag_parameters = {
#     'mag_bias_correlation_time': 1000.0,    # tau_m
#     'mag_noise_density': 0.00001,           # sigma_m
#     'mag_random_walk': 0.000001,            # sigma_b_m
# }


normal_dist = torch.distributions.Normal(0, 1.0) # mean, std_dev

# Time parameters
dt = torch.tensor(0.01, device=device)  # time step
t_end = 10  # total simulation time
time_steps = torch.arange(0, t_end, dt, device=device)

# Storage for results
mag_bias = torch.zeros(3, device=device) 
mag_noise = torch.zeros((len(time_steps), 3), device=device)

# Precompute constants (tau and sigma values)
tau_m = torch.tensor(mag_parameters['mag_bias_correlation_time'], device=device)

sigma_m_d = 1 / torch.sqrt(dt) * mag_parameters['mag_noise_density']
sigma_b_m = mag_parameters['mag_random_walk']
sigma_b_m_d = torch.sqrt(-sigma_b_m**2 * tau_m / 2.0 * (torch.exp(-2.0 * dt / tau_m) - 1.0))
phi_m_d = torch.exp(-1.0 / tau_m * dt)

for idx in range(1, len(time_steps)):
    # Update gyroscope bias and noise (vectorized for all 3 axes)
    mag_bias = phi_m_d * mag_bias + sigma_b_m_d * normal_dist.sample((3,)).to(device)
    mag_noise[idx] = mag_bias + sigma_m_d * normal_dist.sample((3,)).to(device)

# sliding window
window_size = 100
x = mag_noise[1:]
n = x.shape[0]

# Create padded data to handle initial windows
pad_size = window_size - 1
padded_data = torch.cat([x[0].unsqueeze(0).repeat(pad_size, 1), x], dim=0)

# Create sliding windows of equal size
windows = torch.stack([padded_data[i:i+window_size] for i in range(n)])

# Center the data
means = windows.mean(dim=1, keepdim=True)
centered = windows - means

# Calculate covariance matrices
# (batch, window_size, features) -> (batch, features, features)
covs = torch.bmm(centered.transpose(1, 2), centered) / (windows.shape[1] - 1)
# covs = torch.stack([torch.cov(window.T) for window in windows])
 
# Convert back to CPU for plotting
mag_noise = mag_noise.cpu().numpy()
covariance_matrices = covs.cpu().numpy()
time_steps = time_steps.cpu().numpy()

# Plotting the noise for gyroscope and accelerometer
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Magnetometer noise
axs[0].plot(time_steps, mag_noise[:, 0], label='Mag X')
axs[0].plot(time_steps, mag_noise[:, 1], label='Mag Y')
axs[0].plot(time_steps, mag_noise[:, 2], label='Mag Z')
axs[0].set_title('Magnetometer Noise Over Time')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Noise')
axs[0].legend()

# Plot covariance
time_steps_cov = time_steps[1:]  # Adjust time steps for covariance
print(f"covariance_matrices at timestep 0: {covariance_matrices[0]}")
print(f"covariance_matrices at timestep 100: {covariance_matrices[100]}")
print(f"covariance_matrices at timestep 1000: {covariance_matrices[-1]}")

# Diagonal elements (variances)
axs[1].plot(time_steps_cov, covariance_matrices[:, 0, 0], label='Var(X)', linestyle='-')
axs[1].plot(time_steps_cov, covariance_matrices[:, 1, 1], label='Var(Y)', linestyle='-')
axs[1].plot(time_steps_cov, covariance_matrices[:, 2, 2], label='Var(Z)', linestyle='-')

# Off-diagonal elements (covariances)
axs[1].plot(time_steps_cov, covariance_matrices[:, 0, 1], label='Cov(X,Y)', linestyle='--')
axs[1].plot(time_steps_cov, covariance_matrices[:, 0, 2], label='Cov(X,Z)', linestyle='--')
axs[1].plot(time_steps_cov, covariance_matrices[:, 1, 2], label='Cov(Y,Z)', linestyle='--')

axs[1].set_title('Magnetometer Noise Covariance Over Time')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Covariance')
axs[1].legend()

plt.tight_layout()
plt.show()

## 3. Baro Noise Model

In [ ]:
import torch
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Barometer parameters
baro_parameters = {
    'pressure_drift_per_sec': 0.0,
    'pressure_noise_density': 5.0,
    'temperature_noise_density': 1.0,
}
normal_dist = torch.distributions.Normal(0, 0.1) # mean, std_dev

# Time parameters
dt = 0.02  # time step
t_end = 10  # total simulation time
time_steps = torch.arange(0, t_end, dt)

# Storage for results
pressure_drift = 0.0
baro_noise = torch.zeros(len(time_steps))
temp_noise = torch.zeros(len(time_steps))
combined_noise = torch.zeros((len(time_steps), 2))  # Pressure and Temperature
covariance_matrices = []

for idx, t in enumerate(time_steps):
    # Pressure noise and drift
    pressure_drift += baro_parameters['pressure_drift_per_sec'] * dt
    pressure_noise = baro_parameters['pressure_noise_density'] * torch.randn(1).item()
    temp_noise[idx] = baro_parameters['temperature_noise_density'] * torch.randn(1).item()
    
    # Combined noise (pressure + drift)
    baro_noise[idx] = pressure_noise + pressure_drift
    
    # Store combined noise for covariance calculation
    combined_noise[idx] = torch.tensor([baro_noise[idx], temp_noise[idx]])

# sliding window
window_size = 100
x = combined_noise[1:]
n = x.shape[0]

# Create padded data to handle initial windows
pad_size = window_size - 1
padded_data = torch.cat([x[0].unsqueeze(0).repeat(pad_size, 1), x], dim=0)

# Create sliding windows of equal size
windows = torch.stack([padded_data[i:i+window_size] for i in range(n)])

# Center the data
means = windows.mean(dim=1, keepdim=True)
centered = windows - means

# Calculate covariance matrices
# (batch, window_size, features) -> (batch, features, features)
covs = torch.bmm(centered.transpose(1, 2), centered) / (windows.shape[1] - 1)
# covs = torch.stack([torch.cov(window.T) for window in windows])
 
# Convert back to CPU for plotting
baro_noise = baro_noise.cpu().numpy()
temp_noise = temp_noise.cpu().numpy()
covariance_matrices = covs.cpu().numpy()
time_steps = time_steps.cpu().numpy()

# Print covariance matrices at specific timesteps
print(f"Covariance matrix at timestep 0:\n{covariance_matrices[0]}")
print(f"\nCovariance matrix at timestep 100:\n{covariance_matrices[100]}")
print(f"\nCovariance matrix at timestep {len(covariance_matrices)-1}:\n{covariance_matrices[-1]}")

# Plotting
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Plot noise over time
axs[0].plot(time_steps, baro_noise, label='Pressure')
axs[0].plot(time_steps, temp_noise, label='Temperature')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Noise')
axs[0].set_title('Barometer and Temperature Noise Over Time')
axs[0].legend()

# Plot covariance elements
time_steps_cov = time_steps[1:]  # Adjust time steps for covariance plot

# Plot variances
axs[1].plot(time_steps_cov, covariance_matrices[:, 0, 0], 
            label='Var(Pressure)', linestyle='-')
axs[1].plot(time_steps_cov, covariance_matrices[:, 1, 1], 
            label='Var(Temperature)', linestyle='-')
# Plot covariance
axs[1].plot(time_steps_cov, covariance_matrices[:, 0, 1], 
            label='Cov(Pressure,Temp)', linestyle='--')

axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Covariance')
axs[1].set_title('Barometer Noise Covariance Over Time')
axs[1].legend()

plt.tight_layout()
plt.show()

## 4. Sampling rate inconsistency

In [ ]:
import torch
import matplotlib.pyplot as plt

# Set the average sample rate and compute the rate parameter lambda
average_sample_rate = 200  # Hz
lambda_rate = average_sample_rate  # for exponential distribution
min_dt, max_dt = 1/200, 1/195  # dt should vary between 1/200 and 1/195 seconds

# Initialize the exponential distribution with the appropriate rate
exponential_dist = torch.distributions.Exponential(rate=lambda_rate)

# Number of samples to simulate
num_samples = 1000

# Generate random sample intervals from the exponential distribution
dt_samples = exponential_dist.sample((num_samples,))

# Clip the dt values to ensure they stay within the desired range
dt_samples_clipped = torch.clamp(dt_samples, min=min_dt, max=max_dt)

# Convert dt samples to Hz for visualization (1/dt = frequency)
frequencies = 1 / dt_samples_clipped

# Plot the result to visualize the variability in sampling frequency
plt.figure(figsize=(10, 6))
plt.hist(frequencies.numpy(), bins=30, color='c', edgecolor='black', alpha=0.7)
plt.title('Histogram of Sampling Frequency (Hz) with Exponential Distribution')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Count')
plt.grid(True)
plt.show()


In [ ]:
import torch
import torch.distributions as D
import matplotlib.pyplot as plt

# Average sample rate and range
sample_rate_mean = 100.0  # Hz
sample_rate_min = 98.0   # Hz
sample_rate_max = 100.0   # Hz

# Convert sample rates to dt (seconds per sample)
dt_mean = torch.tensor(1.0 / sample_rate_mean)
dt_min = torch.tensor(1.0 / sample_rate_max)  # Minimum dt corresponding to maximum sample rate (200Hz)
dt_max = torch.tensor(1.0 / sample_rate_min)  # Maximum dt corresponding to minimum sample rate (195Hz)

# Estimate log-normal parameters for dt
mu = torch.log(dt_mean)
sigma = (torch.log(dt_max) - torch.log(dt_min)) / 16.0  # Approximation
print(mu, sigma)

# Create a log-normal distribution with the estimated parameters
log_normal_dist = D.LogNormal(mu, sigma)

# Generate dt samples and clip dt values between dt_min and dt_max
num_samples = 1000
dt_samples = torch.clamp(log_normal_dist.sample((num_samples,)), min=dt_min, max=dt_max)

# Plot the dt samples to visualize the distribution
plt.figure(figsize=(8, 5))
plt.hist(dt_samples.numpy(), bins=50, density=True, alpha=0.7, color='green')
plt.axvline(dt_mean, color='red', linestyle='--', label='Mean dt')
#plt.axvline(dt_min, color='blue', linestyle='--', label='dt_min (1/200 Hz)')
plt.axvline(dt_max, color='purple', linestyle='--', label='dt_max (1/195 Hz)')
plt.title('Clipped Log-Normal Distribution of Sampling Time (dt)')
plt.xlabel('dt (seconds)')
plt.ylabel('Probability Density')
plt.legend()
plt.show()

## 5. GPS

In [ ]:
import torch
import torch.distributions as D
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# low noise
gps_parameters = {
    'gps_xy_noise_density': 0.0002,
    'gps_z_noise_density': 0.0004,
    'gps_vxy_noise_density': 0.005,        
    'gps_vz_noise_density': 0.005,     
    'gps_xy_random_walk': 0.05, 
    'gps_z_random_walk': 0.05,        
    'gps_correlation_time': 60.0,      
}

# Initialize biases with turn-on bias using torch distributions
normal_dist = D.Normal(torch.tensor(0.0, device=device), torch.tensor(1.0, device=device))
gps_pos_bias = torch.zeros(3, device=device)
gps_vel_bias = torch.zeros(3, device=device)

# Time parameters
dt = torch.tensor(0.2, device=device)  # time step
t_end = 100  # total simulation time
time_steps = torch.arange(0, t_end, dt, device=device)

# Storage for results
gps_pos_bias = torch.zeros((len(time_steps), 3), device=device)
gps_vel_bias = torch.zeros((len(time_steps), 3), device=device)
gps_pos_noise = torch.zeros((len(time_steps), 3), device=device)
gps_vel_noise = torch.zeros((len(time_steps), 3), device=device)

tau_gps = torch.tensor(gps_parameters['gps_correlation_time'], device=device)
root_dt_inv = 1/ torch.sqrt(dt)
noise_xy = root_dt_inv * torch.tensor(gps_parameters['gps_xy_noise_density'], device=device)
noise_z = root_dt_inv * torch.tensor(gps_parameters['gps_z_noise_density'], device=device)
noise_vxy = root_dt_inv * torch.tensor(gps_parameters['gps_vxy_noise_density'], device=device)
noise_vz = root_dt_inv * torch.tensor(gps_parameters['gps_vz_noise_density'], device=device)
random_walk_gps_xy = root_dt_inv * torch.tensor(gps_parameters['gps_xy_random_walk'], device=device)
random_walk_gps_z = root_dt_inv * torch.tensor(gps_parameters['gps_z_random_walk'], device=device)

noise_gps_pos = torch.tensor([noise_xy, noise_xy, noise_z], device=device)
noise_gps_vel = torch.tensor([noise_vxy, noise_vxy, noise_vz], device=device)
random_walk_gps_pos = torch.tensor([random_walk_gps_xy, random_walk_gps_xy, random_walk_gps_z], device=device)

for idx in range(1, len(time_steps)):
    gps_pos_bias[idx] += (random_walk_gps_pos * normal_dist.sample((3,)).to(device) * dt) - (gps_pos_bias[idx] / tau_gps)
    gps_pos_noise[idx] = gps_pos_bias[idx] + noise_gps_pos * normal_dist.sample((3,)).to(device)

    gps_vel_noise[idx] = noise_gps_vel * normal_dist.sample((3,)).to(device)
    

# Convert back to CPU for plotting

gps_pos_noise = gps_pos_noise.cpu().numpy() 
gps_vel_noise = gps_vel_noise.cpu().numpy()
scaling_position = torch.stack([time_steps, time_steps, torch.zeros(t_end * 5, device=device)], dim=1) * 0.1
pos_noise = gps_pos_noise + scaling_position.cpu().numpy()
time_steps = time_steps.cpu().numpy()

# Plotting the noise for gyroscope and accelerometer
fig, axs = plt.subplots(3, 1, figsize=(10, 8))

# GPS position noise
axs[0].plot(time_steps, gps_pos_noise[:, 0], label='GPS PX')
axs[0].plot(time_steps, gps_pos_noise[:, 1], label='GPS PY')
axs[0].plot(time_steps, gps_pos_noise[:, 2], label='GPS PZ')
axs[0].set_title('GPS Pos Noise Over Time')
axs[0].set_xlabel('Time [s]')
axs[0].set_ylabel('Noise')
axs[0].legend()

# GPS velocity noise
axs[1].plot(time_steps, gps_vel_noise[:, 0], label='GPS VX')
axs[1].plot(time_steps, gps_vel_noise[:, 1], label='GPS VY')
axs[1].plot(time_steps, gps_vel_noise[:, 2], label='GPS VZ')
axs[1].set_title('GPS Vel Noise Over Time')
axs[1].set_xlabel('Time [s]')
axs[1].set_ylabel('Noise')
axs[1].legend()

# Pos with noise
axs[2].plot(time_steps, pos_noise[:, 0], label='X')
axs[2].plot(time_steps, pos_noise[:, 1], label='Y')
axs[2].plot(time_steps, pos_noise[:, 2], label='Z')
axs[2].set_title('Pos with Noise Over Time')
axs[2].set_xlabel('Time [s]')
axs[2].set_ylabel('m')
axs[2].legend()

plt.tight_layout()
plt.show()